In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [2]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    sr: float
    bpb: float  # balls per boundary
    boundary_percent: float
    summary: str

In [3]:
def calculate_strike_rate(state: BatsmanState) -> BatsmanState:
    sr = (state["runs"] / state["balls"]) * 100
    return {'sr': sr}

def calculate_balls_per_boundary(state: BatsmanState) -> BatsmanState:
    total_boundaries = state["fours"] + state["sixes"]
    bpb = state["balls"] / total_boundaries if total_boundaries > 0 else float('inf')
    return {'bpb': bpb}

def calculate_boundary_percentage(state: BatsmanState) -> BatsmanState:
    total_boundary_runs = (state["fours"]*4) + (state["sixes"]*6)
    boundary_percent = total_boundary_runs / state["runs"] * 100 if state["runs"] > 0 else 0
    return {'boundary_percent': boundary_percent}

def summary(state: BatsmanState) -> BatsmanState:
    summary = f"""
    Strike Rate: {state['sr']:.2f}
    Balls per Boundary: {state['bpb']:.2f}
    Boundary Percentage: {state['boundary_percent']:.2f}%
    """
    return {'summary': summary}

In [4]:
graph = StateGraph(BatsmanState)

graph.add_node("Calculate Strike Rate", calculate_strike_rate)
graph.add_node("Calculate Balls Per Boundary", calculate_balls_per_boundary)
graph.add_node("Calculate Boundary Percentage", calculate_boundary_percentage)
graph.add_node("Summary", summary)

#edges
graph.add_edge(START, "Calculate Strike Rate")
graph.add_edge(START, "Calculate Balls Per Boundary")
graph.add_edge(START, "Calculate Boundary Percentage")

graph.add_edge("Calculate Strike Rate", "Summary")
graph.add_edge("Calculate Balls Per Boundary", "Summary")
graph.add_edge("Calculate Boundary Percentage", "Summary")

graph.add_edge("Summary", END)

workflow = graph.compile()

In [5]:
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod
import langchain_core.runnables.graph as graph_mod

# Set the default draw method to Pyppeteer for local rendering
graph_mod.DEFAULT_MERMAID_DRAW_METHOD = MermaidDrawMethod.PYPPETEER

In [9]:
initial_state = {
    "runs": 100,
    "balls": 50,
    "fours": 6,
    "sixes": 4
}

final_state = workflow.invoke(initial_state)

In [10]:
final_state

{'runs': 100,
 'balls': 50,
 'fours': 6,
 'sixes': 4,
 'sr': 200.0,
 'bpb': 5.0,
 'boundary_percent': 48.0,
 'summary': '\n    Strike Rate: 200.00\n    Balls per Boundary: 5.00\n    Boundary Percentage: 48.00%\n    '}